<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

### ARIAtools
The steps in this example have been extracted from the unit tests found in ARIA-tools config.yml, found [here](https://github.com/aria-tools/ARIA-tools/blob/dev/.circleci/config.yml) and the ARIA-tools documentation, found [here](https://github.com/aria-tools/ARIA-tools-docs)

#### Kernel: mintpy

#### Initial set-up

Make sure the ~/.netrc file is updated to include your earthdata login. A line has been inserted in which you substitute your username and password:

    machine urs.earthdata.nasa.gov login <earthdata_user> password <earthdata_password>

This will enable the downloads from the ASF DAAC.

You also need to ensure your AWS credentials are valid.

#### Step 1 - Make a few imports and define the function that performs the plotting

In [ ]:
import os
from osgeo import gdal, ogr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter, StrMethodFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable  
import ARIAtools.shapefile_util as shputil

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    output_dir = os.path.join(start_dir, 'notebook_output/ARIAtools_dem')
    os.makedirs(output_dir, exist_ok=True)
    py_dir = os.path.join(start_dir, 'python')

os.chdir(output_dir)


def plot_layer(path_layer, lay_type=None, cmap=None, **kwargs):
    """ 
        path_layers is a string to the GDAL compatible dataset to be plotted
    """
    
    if not lay_type: 
        lay_type = os.path.dirname(path_layer)
    title = [os.path.basename(lay_type)]
    
    ## get the lon lat bounds
    ds       = gdal.Open(path_layer, gdal.GA_ReadOnly)
    trans    = ds.GetGeoTransform()
    extent   = [trans[0], trans[0] + ds.RasterXSize * trans[1], trans[3] + ds.RasterYSize*trans[5], trans[3]]
    
    ## loading the data
    n_bands  = ds.RasterCount
    lst_arrs = []
    
    for band in range(n_bands):
        raster = ds.GetRasterBand(band+1)
        arr    = raster.ReadAsArray()
        try:
            NoData = raster.GetNoDataValue()
            arr = np.ma.masked_where((arr>1e20) |(arr==NoData),arr )
        except:
            print('Could not find a no-data value...')
            arr = np.ma.masked_where(arr>1e20,arr)
        
        lst_arrs.append(arr)

    ds = None
    if n_bands < 4:
        nrows = 1; ncols = n_bands
    else:
        raise Exception('Number of bands currently unsupported')
        
    
    ## initializing a figure
    fig, axes = plt.subplots(figsize=(12,9), ncols=ncols, nrows=nrows, sharex='col', sharey='row')
    axes = axes if isinstance(axes, np.ndarray) else np.array(axes)
    axe  = axes.ravel() 
    cmap = plt.cm.Greys_r
    cmap.set_under('black')
    
    ## definging the plotting options for differnt layer types
    # Amplitude:
    if lay_type.endswith('amplitude'): 
        # will fix the maximum amplitude bound
        vmin=None
        vmax = 2000 
    # Coherence:
    elif lay_type.endswith('coherence'): 
        # has fixed range between 0-1
        vmin=0
        vmax = 1
    # Incidence angle:
    elif lay_type.endswith('incidenceAngle'): 
        vmin=None
        vmax=None
    # water
    elif lay_type.startswith('water'):
        # no bounds needed will be a 0/1 mask
        vmin=0
        vmax=1
        cmap='Greys'
    # deformation or unwrapped phase
    elif lay_type.startswith('defo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
    elif lay_type.startswith('terr') or lay_type.startswith('topo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.terrain
    elif lay_type == 'ENU':
        vmin=None
        vmax=None
        title = ['East', 'North', 'Up']
        fig.subplots_adjust(wspace=0.5)

    else:
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
        
    # plotting the data    
    for i, ax in enumerate(axe):
        im   = ax.imshow(lst_arrs[i], cmap=cmap, vmin=vmin, vmax=vmax, extent=extent,interpolation='nearest')
        divider = make_axes_locatable(ax)
        cax     = divider.append_axes('right', size='5%', pad=0.25)
        if lay_type == 'ENU':
            fig.colorbar(im, cax=cax, format=FuncFormatter(lambda x, y: '{:.3f}'.format(x)))
        elif lay_type.startswith('water'):
            fig.colorbar(im, cax=cax, ticks=[vmin, vmax])
        else:
            fig.colorbar(im, cax=cax)

        ax.set_title(title[i], fontsize=15)
        ax.grid(False)

    axe[0].set_ylabel('latitude', labelpad=15, fontsize=15)
    axe[int(np.floor(n_bands/2))].set_xlabel('longitude', labelpad=15, fontsize=15)

#### Step 2 : Determine the product(s) from track 42 between 2018/04/02 and 2018/04/14 over the big island.
- Output the count of products
- Output the product URLs

In [ ]:
test_dir = os.path.join(output_dir, 'test2')
prod_dir = os.path.join(test_dir, 'products')
os.makedirs(prod_dir, exist_ok=True)
os.chdir(test_dir)
          
!rm -f products/*.txt

# there should be 6 products
!ariaDownload.py -t 042 -w products --ifg 20180414_20180402 -v -o count

# get the urls of the products
!ariaDownload.py -t 042 -w products --ifg 20180414_20180402 -v -o url

#### Step 3 : Download and process a product from the ASF DAAC

In [ ]:
os.chdir(prod_dir)
!echo Downloading `head -2 download_products_*_0.txt | tail -1` 
!wget --no-clobber --load-cookies /tmp/cookies.txt --save-cookies /tmp/cookies.txt --keep-session-cookies `head -2 download_products_*_0.txt | tail -1`

#### Step 4 : Extract the bounding box from the (first) product file

Make sure your AWS credentials are valid.

#### Notice that we are invoking DEM.py using the isce conda environment! 

In [ ]:
%cd $test_dir

# get the bounding box of the image
!ariaExtract.py -f "products/*.nc"

!cat user_bbox.json

# here we do a bit of black magic to invoke DEM.py (which requires ISCE3) in the isce environment
!conda run -n isce python $py_dir/DEM.py -b `python $py_dir/extract_bbox.py -f user_bbox.json`

#### Step 5 : Perform some extractions of the data; azimuthAngle and amplitude

The first invocation of ariaExtract specifies the NISAR dem file to which a cutline will be applied. Subsequent calls will reuse it.

In [ ]:
%cd $test_dir

# extract azimuthAngle layer
!echo "==========  azimuthAngle  =========="
!ariaExtract.py -f "products/*.nc" -l azimuthAngle -d dem.tif

# list the result files
!echo "Result files"
!ls -1 azimuthAngle

# extract amplitude layer
!echo "==========    amplitude   =========="
!ariaExtract.py -f "products/*.nc" -l amplitude

# list the result files
!echo "Result files"
!ls -1 amplitude

# extract coherehence layer
!echo "==========    coherence   =========="
!ariaExtract.py -f "products/*.nc" -l coherence

# list the result files
!echo "Result files"
!ls -1 coherence

# extract unwrappedPhase layer
!echo "========== unwrappedPhase =========="
!ariaExtract.py -f "products/*.nc" -l unwrappedPhase

# list the result files
!echo "Result files"
!ls -1 unwrappedPhase

#### Step 6 : Show the product bounding box

In [ ]:
!ls -1 productBoundingBox

shputil.plot_shapefile(os.path.join('productBoundingBox','productBoundingBox.json'))

#### Step 7 : Plot the azimuth extraction

In [ ]:
plot_layer(os.path.join('azimuthAngle', '20180414_20180402.vrt'))

#### Step 8 : Plot the amplitude extraction

In [ ]:
plot_layer(os.path.join('amplitude', '20180414_20180402.vrt'))

#### Step 9 : Plot the coherence

In [ ]:
plot_layer(os.path.join('coherence', '20180414_20180402.vrt'))

#### Step 10 : Plot the unwrappedPhase

In [ ]:
plot_layer(os.path.join('unwrappedPhase', '20180414_20180402.vrt'))

#### Step 11: Time-series prep
TBD: Use of watermask switch (i.e. -m download) goes into seemingly non-terminating loop.

In [ ]:
!ariaTSsetup.py -d download -f "products/*.nc"

#### Step 12 : Run loading of aria products in mintpy

In [ ]:
# bit of a kludge to identify where to find prep_aria.py
import mintpy
mintpy_dir = os.path.split(mintpy.__file__)[0]

!python $mintpy_dir/prep_aria.py -s stack/ -d DEM/SRTM_3arcsec.dem -i incidenceAngle/20180414_20180402.vrt -a azimuthAngle/20180414_20180402.vrt

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>